In [2]:
import numpy
import pandas as pd
import seaborn as sns
import random
import math
import os

from IPython import display

pd.set_option('display.max_columns', None)

### Import model objects from their unique directories - easier to organize

from commuter_model.commuter_model import CommuterModel
from electrical_model.electrical_model import ElectricModel

In [2]:
# load ipums data to local variable (ipums_df_19)
ipums_df_19 = pd.read_csv("commuter_model/disaggregated_cleaned_ipums_data.csv",index_col=0)
ipums_df_19 = ipums_df_19[ipums_df_19['YEAR']==2019].reset_index(drop=True).copy()



In [3]:
commuter_model_alltransit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_allmicro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_mix = CommuterModel(ipums_df=ipums_df_19.copy())

In [4]:
### Zeroing out non-transit
commuter_model_alltransit.ipums_df['FLAG_AUTO'] = 0
commuter_model_alltransit.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_alltransit.ipums_df['FLAG_WALK'] = 0
commuter_model_alltransit.ipums_df['FLAG_EBIKE']= 0
commuter_model_alltransit.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_alltransit.ipums_df['FLAG_TAXICAB']= 0

# print(commuter_model_alltransit.ipums_df[['FLAG_AUTO',"FLAG_ESCOOTER","FLAG_WALK","FLAG_EBIKE"]].sum())

#### Transit
### Bus
commuter_model_alltransit.ipums_df['FLAG_EBUSES'] = commuter_model_alltransit.bus_flag_binary(
                                        home_region=commuter_model_alltransit.puma_home_Bus
                                        ,work_region=commuter_model_alltransit.puma_work_Bus
                                        ,schedule=commuter_model_alltransit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_alltransit.ipums_df['FLAG_SUBWAY'] = commuter_model_alltransit.subway_flag_binary(
                                            home_region=commuter_model_alltransit.puma_home_Subway
                                            ,work_region=commuter_model_alltransit.puma_work_Subway
                                            ,schedule=commuter_model_alltransit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_alltransit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_alltransit.commuterRail_flag_binary(
                                             home_region=commuter_model_alltransit.puma_home_CommuterRail
                                            ,work_region=commuter_model_alltransit.puma_work_CommuterRail
                                            ,schedule=commuter_model_alltransit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_alltransit.ipums_df['FLAG_FERRY'] = commuter_model_alltransit.ferry_flag_binary(
                                            home_region=commuter_model_alltransit.puma_home_Ferry
                                           ,work_region=commuter_model_alltransit.puma_work_Ferry
                                           ,schedule=commuter_model_alltransit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Work From Home
commuter_model_alltransit.ipums_df["FLAG_WFH"] = commuter_model_alltransit.wfh_flag_binary(
                                            wfh_dampener=1
                                                )

# print(commuter_model_alltransit.ipums_df[flag_cols].sum())


commuter_model_alltransit.ipums_df['First_Assignment'] = commuter_model_alltransit.RandAssignNoOption(
                                                                commuter_model_alltransit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_alltransit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_alltransit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_alltransit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_alltransit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_alltransit.ipums_df['FLAG_WFH'])
commuter_model_alltransit.ipums_df['TransMode'] = commuter_model_alltransit.RandAssignChosenOption(
                                                                commuter_model_alltransit.ipums_df['First_Assignment'], 
                                                                commuter_model_alltransit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [5]:
flag_cols = [x for x in commuter_model_alltransit.ipums_df if "FLAG" in x]


In [6]:
commuter_model_alltransit.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              6509
dtype: int64

In [7]:
commuter_model_alltransit.ipums_df["First_Assignment"].value_counts()

Subway          9053
Bus             6108
CommuterRail    3879
WFH             2748
No option       1281
Ferry            909
Name: First_Assignment, dtype: int64

In [8]:
commuter_model_alltransit.ipums_df["TransMode"].value_counts()

Subway           9251
Bus              6344
CommuterRail     4157
WFH              2753
Ferry             944
AutoOccupants     467
Walk               40
Other              16
Bicycle             6
Name: TransMode, dtype: int64

In [9]:
commuter_model_alltransit.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              6509
dtype: int64

In [10]:
#### All Micro-Mobility
commuter_model_allmicro.ipums_df['FLAG_AUTO'] = 0
## Motorcycle
commuter_model_allmicro.ipums_df['FLAG_MOTORCYCLE'] = 0
## Taxicab
commuter_model_allmicro.ipums_df['FLAG_TAXICAB'] = 0
## Bus
commuter_model_allmicro.ipums_df['FLAG_EBUSES'] = 0
## Subway
commuter_model_allmicro.ipums_df['FLAG_SUBWAY'] = 0
## Commuter Rail
commuter_model_allmicro.ipums_df['FLAG_COMMUTERRAIL'] = 0
## Ferry
commuter_model_allmicro.ipums_df['FLAG_FERRY'] = 0


## E-Scooter - bump up distance
commuter_model_allmicro.ipums_df['FLAG_ESCOOTER'] = commuter_model_allmicro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_allmicro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_allmicro.ipums_df['FLAG_WALK'] = commuter_model_allmicro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_allmicro.ipums_df['FLAG_EBIKE']=commuter_model_allmicro.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_allmicro.bike_friendly_origins
                                        )


## Work From Home
commuter_model_allmicro.ipums_df["FLAG_WFH"] = commuter_model_allmicro.wfh_flag_binary(
                                            wfh_dampener=1
                                                )

commuter_model_allmicro.ipums_df['First_Assignment'] = commuter_model_allmicro.RandAssignNoOption(
                                                                commuter_model_allmicro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_allmicro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_allmicro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_allmicro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_allmicro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_allmicro.ipums_df['FLAG_WFH'])
commuter_model_allmicro.ipums_df['TransMode'] = commuter_model_allmicro.RandAssignChosenOption(
                                                                commuter_model_allmicro.ipums_df['First_Assignment'], 
                                                                commuter_model_allmicro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [11]:
commuter_model_allmicro.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14140
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES              0
FLAG_SUBWAY              0
FLAG_COMMUTERRAIL        0
FLAG_FERRY               0
FLAG_WFH              6595
dtype: int64

In [12]:
commuter_model_allmicro.ipums_df["First_Assignment"].value_counts(dropna=False)

Bicycle      8553
No option    6778
WFH          4410
Escooter     3850
Walk          387
Name: First_Assignment, dtype: int64

In [13]:
commuter_model_allmicro.ipums_df["TransMode"].value_counts()

Bicycle          8572
WFH              4437
Escooter         3850
AutoOccupants    1785
Subway           1769
CommuterRail     1649
Bus              1216
Walk              485
Ferry             183
Other              32
Name: TransMode, dtype: int64

In [14]:
#### Standard

commuter_model_mix.ipums_df['FLAG_AUTO'] = commuter_model_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )


## Taxicab
commuter_model_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )

## Bus
commuter_model_mix.ipums_df['FLAG_EBUSES'] = commuter_model_mix.bus_flag_binary(
                                        home_region=commuter_model_mix.puma_home_Bus
                                        ,work_region=commuter_model_mix.puma_work_Bus
                                        ,schedule=commuter_model_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_mix.subway_flag_binary(
                                            home_region=commuter_model_mix.puma_home_Subway
                                            ,work_region=commuter_model_mix.puma_work_Subway
                                            ,schedule=commuter_model_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_mix.ipums_df['FLAG_FERRY'] = commuter_model_mix.ferry_flag_binary(
                                            home_region=commuter_model_mix.puma_home_Ferry
                                           ,work_region=commuter_model_mix.puma_work_Ferry
                                           ,schedule=commuter_model_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_mix.bike_friendly_origins
                                            ,male_pct=25
                                            ,female_pct=25
                                            )


## Walking
commuter_model_mix.ipums_df['FLAG_WALK'] = commuter_model_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_mix.ipums_df['FLAG_EBIKE']=commuter_model_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_mix.bike_friendly_origins
                                        )


## Work From Home
commuter_model_mix.ipums_df["FLAG_WFH"] = commuter_model_mix.wfh_flag_binary(
                                            wfh_dampener=.5
                                                )


commuter_model_mix.ipums_df['First_Assignment'] = commuter_model_mix.RandAssignNoOption(
                                                                commuter_model_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_mix.ipums_df['FLAG_WFH'])
commuter_model_mix.ipums_df['TransMode'] = commuter_model_mix.RandAssignChosenOption(
                                                                commuter_model_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [15]:
commuter_model_mix.ipums_df[flag_cols].sum()

FLAG_AUTO             1675
FLAG_ESCOOTER          238
FLAG_WALK             1082
FLAG_EBIKE            7273
FLAG_MOTORCYCLE         30
FLAG_TAXICAB           285
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              3379
dtype: int64

In [16]:
commuter_model_mix.ipums_df["First_Assignment"].value_counts()

Subway           7630
Bus              5376
CommuterRail     3877
Bicycle          2585
WFH              1281
No option        1197
Ferry             877
AutoOccupants     691
Walk              320
Taxicab            74
Escooter           59
Motorcycle         11
Name: First_Assignment, dtype: int64

In [17]:
commuter_model_mix.ipums_df["TransMode"].value_counts()

Subway           7795
Bus              5574
CommuterRail     4208
Bicycle          2589
WFH              1284
AutoOccupants    1109
Ferry             904
Walk              353
Taxicab            74
Escooter           59
Other              18
Motorcycle         11
Name: TransMode, dtype: int64

In [18]:
#### Compare Counts of Flags
# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_1 = commuter_model_allmicro.ipums_df['First_Assignment'].value_counts()

# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_2 = commuter_model_alltransit.ipums_df['First_Assignment'].value_counts()

# commuter_model_mix.ipums_df['FLAG_AUTO'].sum()
ct_3 =commuter_model_mix.ipums_df['First_Assignment'].value_counts()

pd.merge(left=ct_1,right=ct_2,left_index=True,right_index=True,how='outer')\
.merge(right=ct_3,left_index=True,right_index=True,how='outer')

,First_Assignment_x,First_Assignment_y,First_Assignment
AutoOccupants,NaN,NaN,691
Bicycle,8553.0,NaN,2585
Bus,NaN,6108.0,5376
CommuterRail,NaN,3879.0,3877
Escooter,3850.0,NaN,59
Ferry,NaN,909.0,877
Motorcycle,NaN,NaN,11
No option,6778.0,1281.0,1197
Subway,NaN,9053.0,7630
Taxicab,NaN,NaN,74


In [19]:
commuter_model_alltransit.ipums_df.groupby(by=["YEAR"])[flag_cols].sum()
commuter_model_alltransit.ipums_df['First_Assignment'].value_counts()

Subway          9053
Bus             6108
CommuterRail    3879
WFH             2748
No option       1281
Ferry            909
Name: First_Assignment, dtype: int64

In [20]:
master_df = pd.DataFrame()
for x in [commuter_model_alltransit,commuter_model_allmicro,commuter_model_mix]:
    master_df = pd.concat([master_df,x.ipums_df.groupby(by=["TransMode"])['PERWT'].sum()],axis=1)
master_df.columns=["all_transit_count","all_micro_count","mix_count"]
master_df

,all_transit_count,all_micro_count,mix_count
AutoOccupants,48356.0,187565.0,111241.0
Bicycle,436.0,1068354.0,318793.0
Bus,783420.0,133763.0,697723.0
CommuterRail,430532.0,161196.0,430003.0
Ferry,96618.0,17098.0,90777.0
Other,1875.0,3183.0,2068.0
Subway,1143271.0,168409.0,934088.0
WFH,292528.0,461511.0,136241.0
Walk,4294.0,80427.0,59142.0
Escooter,NaN,519824.0,10673.0


In [23]:
save_dir = "../06_Dashboard_Visualization/commuter_model_outputs"
os.makedirs(save_dir,exist_ok=True)
commuter_model_alltransit.ipums_df.to_pickle(f"{save_dir}/commuter_model_alltransit_dfaggregate.pkl")
commuter_model_allmicro.ipums_df.to_pickle(f"{save_dir}/commuter_model_allmicro_dfaggregate.pkl")
commuter_model_mix.ipums_df.to_pickle(f"{save_dir}/commuter_model_mix_dfaggregate.pkl")

In [26]:
subregion = pd.read_csv(f"{save_dir}/subregion_reference_table.csv")
# subregion = subregion.drop_duplicates().reset_index(drop=True)\
#                      .rename({'pumakey':'PUMAKEY_HOME','label':'Subregion'},axis=1)
# subr_dict = {11:'CT',
#              2:'Outer NJ',
#              44:'Manhattan',
#              618:'Inner NJ',
#              5:'Mid Hud',
#              19:'LI',
#              4:'Non-MNY Boroughs',
#              444:'Low Hud'
#             }
# subregion["Subregion"] = subregion["Subregion"].replace(subr_dict)
# subregion.to_csv(f"{save_dir}/subregion_reference_table.csv")
subregion

,Unnamed: 0,PUMAKEY_HOME,Subregion
0,0,09_00102,CT
1,1,09_00101,CT
2,2,09_00100,CT
3,3,09_00103,CT
4,4,09_00105,CT
...,...,...,...
177,177,36_04113,Non-MNY Boroughs
178,178,36_04105,Non-MNY Boroughs
179,179,36_04104,Non-MNY Boroughs
180,180,36_04103,Non-MNY Boroughs


In [27]:
subregion.Subregion.unique()

array(['CT', 'Outer NJ', 'Manhattan', 'Inner NJ', 'Mid Hud', 'LI',
       'Non-MNY Boroughs', 'Low Hud'], dtype=object)